In [1]:
import numpy as np
import scipy.stats as st
import scipy.optimize as opt

# 14.1

In [194]:
def newton_cotes(f, a, b, N, method):
    interval = (b-a)/N
    func = np.vectorize(f)
    x_1 = np.sum(func(np.linspace(a+(0.5*interval),b-(0.5*interval),N)))
    x_2 = np.sum(func(np.linspace(a+interval,b-interval,N-1)))
    if method=='midpoint':
        integral = interval*x_1
    elif method=='trapezoid':
        integral = 0.5*interval*(f(a)+f(b)+2*x_2)
    elif method=='Simpsons':
        integral = (1/6)*interval*(f(a)+f(b)+4*x_1+2*x_2)
    else:
        raise ValueError
    return integral

In [195]:
g = lambda x: 0.1*x**4-1.5*x**3+0.53*x**2+2*x+1
N = 500
A = 4373+(1/3)
print("Midpoint Difference:", newton_cotes(g, -10, 10, N, 'midpoint')-A)
print("Trapezoid Difference:", newton_cotes(g, -10, 10, N, 'trapezoid')-A)
print("Simpsons Difference:", newton_cotes(g, -10, 10, N, 'Simpsons')-A)

Midpoint Difference: -0.0547465173313
Trapezoid Difference: 0.109493162669
Simpsons Difference: 4.26689439337e-08


# 14.2

In [196]:
def newton_norm(mu, sigma, N, k):
    a = mu-sigma*k
    b = mu+sigma*k
    z = np.linspace(a,b,N)
    w = np.empty(N)
    w[0] = st.norm.cdf(0.5*(z[0]+z[1]),loc=mu,scale=sigma)
    w[N-1] = 1-st.norm.cdf(0.5*(z[N-2]+z[N-1]),loc=mu,scale=sigma)
    for i in range(1,N-1):
        zmin = (z[i-1]+z[i])*0.5
        zmax = (z[i+1]+z[i])*0.5
        f = lambda x: st.norm.pdf(z[i],loc=mu,scale=sigma)
        w[i] = newton_cotes(f, zmin, zmax, N, 'Simpsons')
    return z,w

In [197]:
Z, w = newton_norm(0,1,11,2)
print('For N=1, mu=0, and sigma=1:')
print('Nodes:', Z)
print('Weights:', w)

For N=1, mu=0, and sigma=1:
Nodes: [-2.  -1.6 -1.2 -0.8 -0.4  0.   0.4  0.8  1.2  1.6  2. ]
Weights: [ 0.03593032  0.04436833  0.07767442  0.11587662  0.14730806  0.15957691
  0.14730806  0.11587662  0.07767442  0.04436833  0.03593032]


# 14.3

In [198]:
def lognormal(mu, sigma, N, k):
    Z, w = newton_norm(mu, sigma, N, k)
    A = np.exp(Z)
    return A,w
A, w = lognormal(0,1,11,2)
print('For N=1, mu=0, and sigma=1, the lognormal distribution is:')
print('Nodes:', A)
print('Weights:', w)

For N=1, mu=0, and sigma=1, the lognormal distribution is:
Nodes: [ 0.13533528  0.20189652  0.30119421  0.44932896  0.67032005  1.          1.4918247
  2.22554093  3.32011692  4.95303242  7.3890561 ]
Weights: [ 0.03593032  0.04436833  0.07767442  0.11587662  0.14730806  0.15957691
  0.14730806  0.11587662  0.07767442  0.04436833  0.03593032]


# 14.4

In [199]:
def expectation(nodes, weights):
    return np.sum(nodes@weights)
mu = 10.5
sigma = 0.8
N = 100
k = 3
N, W= lognormal(mu, sigma, N, k)
income_expect = expectation(N, W)
actual_expect = np.exp(mu+(sigma**2)/2)
print('Actual Expected Value:', actual_expect)
print('Approximate Expected Value:', income_expect)

Actual Expected Value: 50011.0870085
Approximate Expected Value: 49858.2898986


# 14.5

In [84]:
a = -10
b = 10

def v_func(x):
    vec = np.empty(6)
    for i in range(6):
        j = i+1
        rhs = 0
        for k in range(3):
            rhs += x[k]*(x[k+3]**i)
        vec[i] = (b**j-a**j)/j - rhs
    return vec

x_0 = np.ones(6)
x = opt.root(v_func,x_0,tol=1e-14).x
integral = 0
for k in range(3):
    integral += x[k]*(g(x[k+3]))
actual = 4373+(1/3) 
print('Gaussian Quadrature Difference:', integral-actual)

Gaussian Quadrature Difference: -9.09494701773e-13


# 14.6

In [82]:
from scipy.integrate import quad as quad
scipy_estimate = quad(g,a,b)[0]
print('Scipy Quadrature Difference:', scipy_estimate-actual)

Scipy Quadrature Difference: 9.094947017729282e-13


# 14.7

In [103]:
g = lambda x: 1 if x[0]**2+x[1]**2 <= 1 else 0
omega = np.array([[-1,1],[-1,1]])
def monte_carlo_int(g,omega,N):
    m,n = omega.shape
    draws = np.empty((N,n))
    summation = 0
    for i in range(N):
        draws[i,:] = np.random.uniform(omega[0,0],omega[0,1],n)
        summation+=g(draws[i,:])
    return (4/N)*summation
N = int(1e7)
print('Approx of pi using monte carlo:', monte_carlo_int(g,omega,N))

Approx of pi using monte carlo: 3.1424108


As seen, the Monte Carlo method requires an extremely large number of draws.

# 14.8

In [2]:
from sympy import sieve

# Generate nth rational number using binary representation
def leaf2frac(leaf,m=0,n=1):
    for bit in leaf:
        if bit=='0': n += m
        if bit=='1': m += n
    return m/n
    
def equidistribute(n,d,sequence):
    if sequence=='Weyl':
        sieve.extend_to_no(d)
        primes = np.array(sieve._list)[:d]
        element = np.modf(n*(primes**0.5))[0]
    elif sequence=='Haber':
        sieve.extend_to_no(d)
        primes = np.array(sieve._list)[:d]
        s = 0.5*n*(n+1)
        element = np.modf(s*(primes**0.5))[0]
    elif sequence=='Niederreiter':
        powers = np.linspace(1,d,d)/np.linspace(2,d+1,d)
        element = np.modf(n*((2*np.ones(d))**powers))[0]
    elif sequence=='Baker':
        element = np.empty(d)
        for i in range(d):
            leaf = format(i+1,'b')
            element[i] = n*np.exp(leaf2frac(leaf))
        element = np.modf(element)[0]
    else:
        raise ValueError
    return element

# 14.9

In [3]:
g = lambda x: 1 if x[0]**2+x[1]**2 <= 1 else 0
omega = np.array([[-1,1],[-1,1]])
def quasi_monte_carlo(g,omega,N,method):
    m,d = omega.shape
    draws = np.empty((N,d))
    summation = 0
    low = omega[0,0]
    high = omega[0,1]
    interval = high-low
    for i in range(N):
        draws[i,:]=low+interval*equidistribute(i,d,method)
        summation+=g(draws[i,:])
    return (4/N)*summation

N = int(1e6)
print('Approx of pi using Weyl monte carlo:', quasi_monte_carlo(g,omega,N,'Weyl'))
print('Approx of pi using Haber monte carlo:', quasi_monte_carlo(g,omega,N,'Haber'))
print('Approx of pi using Niederreiter monte carlo:', quasi_monte_carlo(g,omega,N,'Niederreiter'))
print('Approx of pi using Baker monte carlo:', quasi_monte_carlo(g,omega,N,'Baker'))


Approx of pi using Weyl monte carlo: 3.141364
Approx of pi using Haber monte carlo: 3.1374359999999997
Approx of pi using Niederreiter monte carlo: 3.141768
Approx of pi using Baker monte carlo: 3.1415919999999997


As seen, the rate of convergence for the quasi-Monte Carlo integration seems to converge faster towards pi than for pseudorandom Monte Carlo simulation. That being said, it appears that Baker has the most accurate approximation.